In [1]:
import pandas as pd
from stats_can import StatsCan

# Database connector 
from sqlalchemy import create_engine

In [2]:
# MySQL connection details
username = 'root'
password = '**'
host = 'localhost'
port = '3306'  
database = 'gasdb'

# Create SQLAlchemy engine for MySQL connection
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}')

In [3]:
# Import vector csv
vector_df = pd.read_csv(r'./Data/Vector Data.csv')

In [4]:
df = vector_df
df['geo_vector'] = df['Geography'] + '_' + df['GasTypeID'].astype(str)

# store in a dictionary 
dict_map = dict(zip(df['geo_vector'], df['VectorCode']))
v_list = list(dict_map.values())

# get the data from statistics canada
sc = StatsCan()
v_df = sc.vectors_to_df(v_list)

inv_map = {v: k for k, v in dict_map.items()}
v_df.columns = v_df.columns.to_series().map(inv_map)
v_df.index.names = ['Date']
final_df = v_df.reset_index()

# unpivot the dataframe
a_melted = pd.melt(final_df, id_vars=['Date'], var_name='GEO', value_name='Value')

# Rearrange columns for the desired output
df_final = a_melted[['Date', 'GEO', 'Value']]

# Display the result
new_df = df_final.sort_values(by=['Date','GEO'])

# Split the GEO columns
new_df['Gas_Type'] = new_df['GEO'].str.split('_').str[-1]
# convert 'gas_type' to integer
new_df['Gas_Type'] = new_df['Gas_Type'].astype(int)
new_df['GEO'] = new_df['GEO'].str.split('_').str[0]
new_df.tail()


c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\stats_can\api_class.py:24: FutureWarning: This class will be deprecated in upcoming v3 release. Please see the docs for details
  warn(
c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\stats_can\sc.py:685: FutureWarning: This function will be deprecated in the v3 release. Please see the docs for details.
  warn(
c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\stats_can\sc.py:360: FutureWarning: This function will be deprecated in the v3 release. Please see the docs for details.
  warn(
c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\stats_can\sc.py:276: FutureWarning: This function will be deprecated in the v3 release. Please see the docs for details.
  warn(
c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\stats_can\sc.py:326: FutureWarning: This function will be deprecated in the v3 release. Please see the docs for 

,Date,GEO,Value,Gas_Type
11647,2024-08-01,"Winnipeg, Manitoba",171.3,2
12063,2024-08-01,"Winnipeg, Manitoba",161.1,3
21215,2024-08-01,"Yellowknife, Northwest Territories",162.9,1
21631,2024-08-01,"Yellowknife, Northwest Territories",184.4,2
22047,2024-08-01,"Yellowknife, Northwest Territories",180.5,3


In [13]:
# Export to database
# Insert DataFrame into MySQL table
table_name = 'prices'

# Insert data into the table, if the table already exists, replace the data
new_df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

print("Data inserted successfully!")

Data inserted successfully!
